In [1]:
import json
from collections import defaultdict
from itertools import permutations

In [2]:
with open('talent_profiles_20k.json') as f:
    data = json.load(f)

In [3]:
len(data)

20000

# Basic idea

### The idea is this: "two skills or titles are relates/similar if they often happen together across the candidates". This in essense is like a NN classifier.
### Here, I am not much concerned with the word processing (stemming, cleaning some special characters, etc) but that is something that can be done to improve the results.


# Most related skills

### 1) If input a list of skills, return the top-10 most related or similar skills;

In [4]:
pair_freq = dict()
for candidate in data:
    skills = list(set(map(lambda s:s.lower().strip(), data[candidate]['basic']['skill'])))
    
    for paird_skill in permutations(skills,2):
        if paird_skill in pair_freq:
            pair_freq[paird_skill] +=  1
        else:
            pair_freq[paird_skill] =  1     

In [5]:
similar_skills = defaultdict(int)
for (pair_a, pair_b), val in pair_freq.items():
    if not pair_a in similar_skills:
        similar_skills[pair_a] = dict()
    else:
        similar_skills[pair_a][pair_b] = val

In [6]:
# test time
test_skills = ['sql', 'javascript', 'keras', 'pytorch']

for skill in test_skills:
    print(f"\n\nmost related skills to skill \"{skill}\":")
    tmp_dict = similar_skills[skill]
    if not similar_skills[skill]:
        print('no match found')
    else:
        tmp_dict = dict(sorted(tmp_dict.items(), key=lambda item: item[1], reverse=True))
        for key,val in tuple(tmp_dict.items())[:10]:
            print(f"{key}: {val}")



most related skills to skill "sql":
java: 1057
javascript: 962
software development: 842
html: 759
microsoft sql server: 689
xml: 672
c#: 664
microsoft office: 603
databases: 591
mysql: 572


most related skills to skill "javascript":
sql: 962
java: 934
html: 922
css: 821
jquery: 668
c#: 665
software development: 643
mysql: 637
xml: 595
php: 557


most related skills to skill "keras":
deep learning: 2
python: 2
tensorflow: 2
git: 2
machine learning: 2
matlab: 2
predictive analytics: 1
data: 1
bluetooth: 1
ros: 1


most related skills to skill "pytorch":
no match found


# Most related titles

### 2) If input a list of titles, return the top-10 most related or similar titles;

In [7]:
pair_freq_titles = dict()
for candidate in data:
    if not data[candidate]['position']:
        continue
    positions = []
    for pos_t in data[candidate]['position']:
        positions.append(pos_t['position_title'])

    titles = [s.lower() for s in positions if s]
    titles = list(set(titles))
    
    for paird_title in permutations(titles,2):
        if paird_title in pair_freq_titles:
            pair_freq_titles[paird_title] +=  1
        else:
            pair_freq_titles[paird_title] =  1 
    

In [9]:
similar_titles = defaultdict(int)
for (pair_a, pair_b), val in pair_freq_titles.items():
    if not pair_a in similar_titles:
        similar_titles[pair_a] = dict()
    else:
        similar_titles[pair_a][pair_b] = val


In [10]:
# test time
test_titles = ['programmer', 'ceo', 'intern', 'consultant', 'student']

for title in test_titles:
    print(f"most related titles to the title \"{title}\":\n")
    tmp_dict = similar_titles[title]
    if not similar_titles[title]:
        print('no match found')
    else:
        tmp_dict = dict(sorted(tmp_dict.items(), key=lambda item: item[1], reverse=True))
        for key,val in tuple(tmp_dict.items())[:10]:
            print(f"{key}: {val}")
            
    print("\n\n")        

most related titles to the title "programmer":

software developer: 12
software engineer: 10
web developer: 7
senior developer: 6
senior programmer: 5
senior software engineer: 4
developer: 3
senior consultant: 3
it specialist: 3
project manager: 3



most related titles to the title "ceo":

owner: 13
president: 13
founder: 12
managing director: 11
director: 10
partner: 8
project manager: 5
executive vice president: 4
coo: 4
customer service representative: 3



most related titles to the title "intern":

software engineer: 20
research assistant: 17
consultant: 15
project manager: 15
sales associate: 14
account executive: 13
investment banking analyst: 11
student: 11
software developer: 11
associate: 10



most related titles to the title "consultant":

senior consultant: 58
manager: 28
analyst: 22
project manager: 18
intern: 15
software engineer: 15
associate consultant: 14
director: 11
business analyst: 11
research assistant: 10



most related titles to the title "student":

intern:

# Most related candidates

### 3) Bonus question: if input is a list of skills or a list of titles, return the top-10 most related candidate profiles.

In [12]:
desired_skills = ['java', 'sql', 'xml', 'jquery']
desired_titles = ['ceo', 'manager']

short_list_skills = dict()
short_list_titles = dict()

for candidate in data:
    skills = [1 for skill in desired_skills if skill.lower() in data[candidate]['basic']['skill']]
    short_list_skills[candidate] = sum(skills)
    
    if data[candidate].get('position'):
        candidate_positions = [x['position_title'].lower() for x in data[candidate]['position'] if x.get('position_title')]
        titles = [1 for position in desired_titles if position.lower() in candidate_positions]
        short_list_titles[candidate] = sum(titles)


In [13]:
print(f'most related candidates based on skills {desired_skills}:\n')

if not short_list_skills:
    print('no match found')
else:
    tmp_dict = dict(sorted(short_list_skills.items(), key=lambda item: item[1], reverse=True))
    for key,val in tuple(tmp_dict.items())[:10]:
        print(f"{key}: {val}")

print("\n\n")

most related candidates based on skills ['java', 'sql', 'xml', 'jquery']:

fu592723926f8529.08526213: 4
fu59270420aebe09.95361160: 4
fu5926f2a166fac6.43957966: 4
fu5925f95874c738.48069279: 4
fu5926e524019802.33670393: 4
fu5926b705302e71.37102698: 4
fu59267fe2462c54.30700212: 4
fu5926e824040a60.75838379: 4
fu5927061eee43d4.91308986: 4
fu592685404ca3c1.69125661: 4





In [14]:
print(f'most related candidates based on titles {desired_titles}:\n')

if not short_list_titles:
    print('no match found')
else:
    tmp_dict = dict(sorted(short_list_titles.items(), key=lambda item: item[1], reverse=True))
    for key,val in tuple(tmp_dict.items())[:10]:
        print(f"{key}: {val}")

print("\n\n")

most related candidates based on titles ['ceo', 'manager']:

fu5927162b89fe49.25949472: 2
fu59269d99e5c877.55548570: 1
fu59270581e016f3.44208976: 1
fu5926ea0f8151b1.89219598: 1
fu592655e16ca722.67065826: 1
fu59270d926c9ec1.13174876: 1
fu5926e855e27731.12873751: 1
fu5926ed7e187f19.10950012: 1
fu592603fb8f3743.56515102: 1
fu5926d85424b211.18952320: 1





In [291]:
# short_list_skills
# data[candidate]['position']
pair_freq_titles

{('developer & help desk specialist', 'senior software developer'): 1,
 ('senior software developer', 'developer & help desk specialist'): 1,
 ('programmer', 'barista'): 1,
 ('programmer', 'shift supervisor'): 1,
 ('barista', 'programmer'): 1,
 ('barista', 'shift supervisor'): 2,
 ('shift supervisor', 'programmer'): 1,
 ('shift supervisor', 'barista'): 2,
 ('director, tr3s research and consumer insights',
  'director, international research & insights, vimn'): 1,
 ('director, tr3s research and consumer insights',
  'connections associate director'): 1,
 ('director, tr3s research and consumer insights',
  'multicultural planning supervisor'): 1,
 ('director, tr3s research and consumer insights', 'media planner'): 1,
 ('director, tr3s research and consumer insights', 'exhibits manager'): 1,
 ('director, tr3s research and consumer insights', 'marketing'): 1,
 ('director, international research & insights, vimn',
  'director, tr3s research and consumer insights'): 1,
 ('director, internati